## Build Prediction pipeline

In [347]:
import pandas as pd
import numpy as np
import pickle

path = '/Users/Mengying/Desktop/QA_Google/data/'

### Step 1: long df prediction in Google Cloud Platform
Does the paragraph contain the answer to the question?

### Step 2: Limit each example id one long answer

In [248]:
long_df_test = pd.read_csv(path+"long_df_sub.csv")
submission = pd.read_csv(path+"sample_submission.csv")

In [250]:
# process submission file for later merge
submission[['id','type']] = submission.example_id.str.split("_",expand=True)
submission.head(1)

,example_id,PredictionString,id,type
0,-1011141123527297803_long,NaN,-1011141123527297803,long


In [251]:
# limit one answer per id, keeping the maximum probability one, flagged under "flag" column
long_df_test["max_prob"] = long_df_test.groupby(["example_id"])['is_answer'].transform(max)
long_df_test["flag"] = 0
long_df_test.loc[long_df_test["max_prob"] == long_df_test["is_answer"], "flag"] =  1

In [252]:
# keep only flag == 1
long_df_test = long_df_test[long_df_test.flag == 1]

In [253]:
# rename to match submission file column name and specify answer type
long_df_test = long_df_test.rename(columns={"example_id": "id"})
long_df_test['type'] = 'long'

In [254]:
long_df_test.head()#.loc[long_df_test.id == -1220107454853145579]

,paragraph,question,is_answer,id,bigrams,PredictionString,max_prob,flag,type
0,"<Table> <Tr> <Th_colspan=""2""> High Commission ...",who is the south african high commissioner in ...,0.969843,-1220107454853145579,2,18:136,0.969843,1,long
10,"<Table> <Tr> <Th_colspan=""2""> `` Michael 's La...",the office episode when they sing to michael,0.971829,8777415633185303067,2,23:190,0.971829,1,long
15,<P> The Cross of Gold speech was delivered by ...,what is the main idea of the cross of gold speech,0.983355,4640548859154538040,5,165:272,0.983355,1,long
58,"<P> Bard was born in Chorlton - cum - Hardy , ...",when was i want to sing in opera written,0.932635,-5316095317154496261,5,201:374,0.932635,1,long
73,<P> The concept of Collision Course was deeply...,who does the voices in ice age collision course,0.975164,-8752372642178983917,3,1796:1943,0.975164,1,long


In [256]:
# filter out those who have less than threshold is_answer probability
threshold = 0.598
long_df_test = long_df_test[long_df_test.is_answer >= threshold]
#long_df_test.is_answer.describe()

In [257]:
# Type conversion
submission.loc[:,'id'] = submission['id'].astype(str)
long_df_test.loc[:,'id'] = long_df_test['id'].astype(str)
# submission.id.dtypes
# long_df_test.type.dtypes

In [258]:
# now merge to the submission file, whichever didn't get merged(and thus PredictionString remains
# NaN, are those that bigram score = 0, or filtered out because of low is_answer prob)
submission = pd.merge(submission[["example_id","id","type"]],long_df_test, on=['id','type'], how="left")
submission.head(2)

,example_id,id,type,paragraph,question,is_answer,bigrams,PredictionString,max_prob,flag
0,-1011141123527297803_long,-1011141123527297803,long,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1011141123527297803_short,-1011141123527297803,short,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:
len(submission[(submission.is_answer.isnull()) &(submission.type=="long")])

25

In [259]:
submission.loc[submission.id == '-8752372642178983917']

,example_id,id,type,paragraph,question,is_answer,bigrams,PredictionString,max_prob,flag
312,-8752372642178983917_long,-8752372642178983917,long,<P> The concept of Collision Course was deeply...,who does the voices in ice age collision course,0.975164,3.0,1796:1943,0.975164,1.0
313,-8752372642178983917_short,-8752372642178983917,short,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
final = submission[['example_id','PredictionString']]
final.head(20)

,example_id,PredictionString
0,-1011141123527297803_long,NaN
1,-1011141123527297803_short,NaN
2,-1028916936938579349_long,42:321
3,-1028916936938579349_short,NaN
4,-1055197305756217938_long,1288:1510
5,-1055197305756217938_short,NaN
6,-1074129516932871805_long,2741:2954
7,-1074129516932871805_short,NaN
8,-1114334749483663139_long,2696:2804
9,-1114334749483663139_short,NaN


AttributeError: 'list' object has no attribute 'isin'

In [239]:
final.to_csv(path+"final.csv",index=False)

In [241]:
final = pd.read_csv(path+"final.csv")

In [243]:
final.head(20)

,example_id,PredictionString
0,-1011141123527297803_long,NaN
1,-1011141123527297803_short,NaN
2,-1028916936938579349_long,42:321
3,-1028916936938579349_short,NaN
4,-1055197305756217938_long,1288:1510
5,-1055197305756217938_short,NaN
6,-1074129516932871805_long,2741:2954
7,-1074129516932871805_short,NaN
8,-1114334749483663139_long,2696:2804
9,-1114334749483663139_short,NaN


### Step 3: filter and keep those whose long answer candidate contains the answer and pass into short df model to obtain short answer, if above threshold tuned by development set.

In [311]:
test = pd.read_csv(path+"/yesno_test_pred.csv")

In [264]:
# performance
threshold = 1
print("TP: ", test[(test.pred==1) & (test.is_answer=="YES")].shape[0]/test[test.is_answer=="YES"].shape[0])
print("FP: ", test[(test.pred==1) & (test.is_answer=="NO")].shape[0]/test[test.is_answer=="NO"].shape[0])
print("Accuracy: ",test[(test.pred==1) & (test.is_answer=="YES") | (test.pred==0) & (test.is_answer=="NO") ].shape[0]/test.shape[0])
# missed (false negative)
print("missed: ", test[(test.pred==0) & (test.is_answer=="YES")].shape[0]/test[test.is_answer=="YES"].shape[0])

print(test[test.is_answer=="YES"].shape[0]/test.shape[0])
print(test[test.pred==1].shape[0]/test.shape[0])


TP:  0.963855421686747
FP:  0.8848484848484849
Accuracy:  0.6564215148188803
missed:  0.03614457831325301
0.6377607025246982
0.9352360043907794


In [341]:
# performance
threshold = 0.5
print("TP: ", test[(test.pred>threshold) & (test.is_answer=="YES")].shape[0]/test[test.is_answer=="YES"].shape[0])
print("FP: ", test[(test.pred<=threshold) & (test.is_answer=="NO")].shape[0]/test[test.is_answer=="NO"].shape[0])
print("Accuracy: ",test[(test.pred>threshold) & (test.is_answer=="YES") | (test.pred==0) & (test.is_answer=="NO") ].shape[0]/test.shape[0])
# missed (false negative)
print("missed: ", test[(test.pred<=threshold) & (test.is_answer=="YES")].shape[0]/test[test.is_answer=="YES"].shape[0])

print(test[test.is_answer=="YES"].shape[0]/test.shape[0])
print(test[test.pred>threshold].shape[0]/test.shape[0])


TP:  0.963855421686747
FP:  0.11515151515151516
Accuracy:  0.6147091108671789
missed:  0.03614457831325301
0.6377607025246982
0.9352360043907794


In [335]:
# Create the dictionary 
event_dictionary ={'YES' : 1, 'NO' : 0} 
  
# Add a new column named 'Price' 
test['is_answer_flag'] =  test['is_answer'].map(event_dictionary) 

In [345]:
index=11
print(test[(test.pred>0.5) & (test.is_answer=="NO")].iloc[index,0])
print(test[(test.pred>0.5) & (test.is_answer=="NO")].iloc[index,1])
print("prob: ", test[(test.pred>0.5) & (test.is_answer=="NO")].iloc[index,4])
test[(test.pred>0.5) & (test.is_answer=="NO")].iloc[index,2]


<P> Lincoln Burrows , played by Dominic Purcell , is a fictional character and one of the two protagonists of the American television series Prison Break . The plot of Prison Break revolves around Lincoln Burrows ' setup for the murder of the vice president 's brother and his brother 's plan to help him escape his death sentence . Lincoln 's brother , Michael Scofield is portrayed by Wentworth Miller . In episode flashbacks , the teenage Lincoln is played by Max Kirsch , while a younger Michael is portrayed by Dylan Minnette . Lincoln Burrows was born on March 17 , 1970 . </P>
are michael and lincoln brothers in real life
prob:  0.98899066


'NO'

In [316]:
test.pred.describe()

count    911.000000
mean       0.849794
std        0.200298
min        0.005672
25%        0.834214
50%        0.921944
75%        0.965892
max        0.997788
Name: pred, dtype: float64

In [296]:
test[(test.pred==1) & (test.is_answer=="NO")].iloc[7,3]

1654978479886221212

In [310]:
test.head()

,paragraph,question,is_answer,example_id,pred
0,<P> During Microsoft 's E3 2015 press conferen...,will an xbox 360 game work on an xbox one,YES,-5000357343921088794,1
1,"<Table> Subway Series <Tr> <Td_colspan=""2""> Th...",have the mets and yankees ever played in a wor...,YES,-8971084966739055955,1
2,"<P> Paraffin wax is a soft colourless solid , ...",is candle wax a solid at room temperature,YES,2059978665782122354,1
3,<P> A blue rose is a flower of the genus Rosa ...,is there such a thing as a blue rose,YES,6704030183739257824,1
4,<P> The Washington Monument is an obelisk on t...,is the washington monument the tallest monumen...,YES,6001942272464482925,1


### short df inspection

In [354]:
with open(path+"/test_short_df_pred.pkl",'rb') as f:
    test_pred = pickle.load(f)

with open(path+"/test_short_df_tokens.pkl",'rb') as f:
    test_tokens = pickle.load(f)

test = pd.read_csv(path+"test_short_df.csv")

In [355]:
test.head()

,paragraph,question,is_answer,example_id
0,<Table> A Bathing Ape ア ・ ベイジング ・ エイプ ( Bape )...,who is the owner of the brand bape,I.T Group,-650067327226916973
1,<P> Whereas an abbreviation may be any type of...,when the letters of a word stand for something,acronym,-2260771055326421759
2,<P> Dubai ( / duːˈbaɪ / doo - BY ; Arabic : دب...,what is the name of the sea surrounding dubai,Persian Gulf,108744506445139712
3,"<P> During rehearsals , the stage manager typi...",who is responsible for organising the rehearsa...,The stage manager,8151674051120688132
4,<P> The Queen 's Guard and Queen 's Life Guard...,what is the purpose of the royal guards,infantry and cavalry soldiers charged with gua...,-301625219284024039


In [393]:
pairs = []

for entry in range(199):
    entry = test_pred[entry,:,:]
    score = {}
    context_start = test_tokens[1].index('[SEP]')
    
    # 2*128
    start = entry[0,:]
    end = entry[1,:]
    for i in range(128):
        if i <= context_start:
            continue
        for j in range(128):
            if j < i: continue
            score[(i,j)] = start[i] + end[j]
    score = {k: v for k, v in sorted(score.items(), key=lambda item: item[1],reverse=True)}
    pairs.append(score)        
        

In [515]:
pairs[8]

{(69, 73): -0.2705169,
 (35, 73): -2.4004169,
 (33, 73): -3.101068,
 (60, 73): -3.5538974,
 (70, 73): -4.818501,
 (72, 73): -5.052945,
 (69, 80): -5.7586346,
 (73, 73): -5.863841,
 (35, 39): -6.3778434,
 (34, 73): -6.610053,
 (35, 41): -6.6628237,
 (69, 71): -6.876916,
 (25, 73): -7.0502486,
 (48, 73): -7.055117,
 (33, 39): -7.0784945,
 (17, 73): -7.1166453,
 (33, 41): -7.363475,
 (41, 73): -7.413936,
 (69, 79): -7.5707746,
 (39, 73): -7.6431055,
 (36, 73): -7.6443467,
 (71, 73): -7.6877933,
 (35, 80): -7.8885345,
 (69, 70): -8.091317,
 (35, 62): -8.138302,
 (69, 90): -8.45461,
 (33, 80): -8.589186,
 (33, 62): -8.838953,
 (40, 73): -8.871147,
 (35, 71): -9.006816,
 (60, 80): -9.042015,
 (37, 73): -9.195833,
 (69, 77): -9.280986,
 (60, 62): -9.291782,
 (69, 75): -9.293801,
 (69, 72): -9.340718,
 (35, 53): -9.440111,
 (69, 69): -9.461753,
 (69, 98): -9.615211,
 (35, 79): -9.700674,
 (33, 71): -9.707467,
 (35, 40): -9.748934,
 (61, 73): -10.128987,
 (33, 53): -10.140762,
 (60, 71): -10.16

In [519]:
test_tokens[8][69:80]

['jose',
 'would',
 'be',
 'the',
 'new',
 'state',
 "'",
 's',
 'capital',
 '.',
 'after']

In [517]:
index = 8
test.iloc[index,0]

"<P> The California State Legislature , with the support of Governor John Bigler , moved to Sacramento in 1854 . The capital of California under Spanish ( and , subsequently , Mexican ) rule had been Monterey , where in 1849 the first Constitutional Convention and state elections were held . The convention decided that San Jose would be the new state 's capital . After 1850 , when California 's statehood was ratified , the legislature met in San Jose until 1851 , Vallejo in 1852 , and Benicia in 1853 , before moving to Sacramento . In the Sacramento Constitutional Convention of 1879 , Sacramento was named to be the permanent state capital . </P>"

In [518]:
print(test.iloc[index,1])
print(test.iloc[index,2])

when did sacramento became the capital of california
1879


In [509]:
#test = test[(test.is_answer!="YES")&(test.is_answer!="NO")]

In [501]:
test_tokens[index]

['[CLS]',
 'who',
 'created',
 'the',
 'first',
 'national',
 'park',
 'in',
 'america',
 '[SEP]',
 '<',
 'p',
 '>',
 'the',
 'national',
 'park',
 'idea',
 'has',
 'been',
 'credited',
 'to',
 'the',
 'artist',
 'george',
 'cat',
 '##lin',
 '.',
 'in',
 '1832',
 'he',
 'traveled',
 'the',
 'northern',
 'great',
 'plains',
 'of',
 'the',
 'united',
 'states',
 ',',
 'where',
 'he',
 'became',
 'concerned',
 'about',
 'the',
 'destruction',
 'of',
 'the',
 'indian',
 'civilization',
 ',',
 'wildlife',
 ',',
 'and',
 'wilderness',
 'as',
 'eastern',
 'settlements',
 'spread',
 'westward',
 '.',
 'he',
 'wrote',
 ',',
 '`',
 '`',
 'by',
 'some',
 'great',
 'protecting',
 'policy',
 'of',
 'government',
 '.',
 '.',
 '.',
 'in',
 'a',
 'magnificent',
 'park',
 '.',
 '.',
 '.',
 'a',
 'nation',
 "'",
 's',
 'park',
 ',',
 'containing',
 'man',
 'and',
 'beast',
 ',',
 'in',
 'all',
 'the',
 'wild',
 '(',
 'ness',
 ')',
 'and',
 'fresh',
 '##ness',
 'of',
 'their',
 'nature',
 "'",
 's',
 'be

In [356]:
test_pred.argmax(-1)[1,]

array([35, 34])

## For submission kernel

In [ ]:
with open('/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl', 'r') as f:
    for i, line in enumerate(f):
        parsed_line = json.loads(line)
        chunk_test_df = pd.DataFrame.from_records([parsed_line], index=[0])
        submission_chunks = submission_chunks + [generate_submission(chunk_test_df)]  # redefine over append